In [95]:
from delphin import ace, mrs
from delphin.codecs import simplemrs
from pogg.pogg_config import POGGConfig
import pogg.my_delphin.sementcodecs as sementcodecs
import pogg.semantic_composition.sement_util as sement_util
import pogg.semantic_composition.semantic_algebra as sem_algebra

pogg_config = POGGConfig("data/config.yml")

def check_if_quantified(check_SEMENT):
    """
    Check if the given SEMENT is quantified (assumes generation is occurring on referring expressions)
    :param check_SEMENT: SEMENT to be checked
    :type check_SEMENT: SEMENT
    :return: quantified SEMENT (may be unchanged from given)
    :rtype: SEMENT
    """
    # if the INDEX (or something eq to INDEX) is not the ARG0 of something with RSTR, gg
    index = check_SEMENT.index
    index_set = set()
    index_set.add(index)
    # go through eqs to find variables eq to index
    if check_SEMENT.eqs is not None:
        for eq in check_SEMENT.eqs:
            if index in eq:
                for elem in eq:
                    index_set.add(elem)

    for rel in check_SEMENT.rels:
        if rel.args['ARG0'] in index_set and 'RSTR' in rel.args:
            return True
    return False

def print_and_generate(sement):
    pre_wrapped_sement = sement_util.overwrite_eqs(sement)

    print("PRE-WRAPPED MRS...")
    mrs_string = simplemrs.encode(pre_wrapped_sement, indent=True)
    print("{}\n".format(mrs_string))

    print("WRAPPED & QUANTIFIED MRS...")
    unknown = sem_algebra.create_base_SEMENT(pogg_config, "unknown")
    # check if INDEX is of type e, wrap it if not
    if sement.index[0] != "e":
        if not check_if_quantified(sement):
            udef = sem_algebra.create_base_SEMENT(pogg_config, "udef_q")
            quantified_sement = sem_algebra.op_scopal_quantifier(udef, sement)
        else:
            quantified_sement = sement

        # plug ARG of unknown with INDEX
        new_sement = sem_algebra.op_non_scopal_functor_hook(unknown, quantified_sement, "ARG")
    else:
        new_sement = sement
    # qeq between new imaginary GTOP and LTOP of sement
    new_sement.hcons.append(mrs.HCons("h000", "qeq", sement.top))
    new_sement.top = "h000"

    # overwrite_eqs after composition
    wrapped_sement = sement_util.overwrite_eqs(new_sement)

    wrapped_mrs_string = simplemrs.encode(wrapped_sement, indent=True)
    print("{}\n".format(wrapped_mrs_string))


    # GENERATOR
    with ace.ACEGenerator(pogg_config.grammar_location, ['-r', 'root_frag']) as generator:
            generator_response = generator.interact(wrapped_mrs_string)
            print("GENERATED RESULTS...")
            for r in generator_response.results():
                print(r.get('surface'))
    print("\n\n\n")

/Users/lizcconrad/.venvs/pogg_tests/lib/python3.13/site-packages/delphin/semi.py:471: SemIWarning: _be_v_id: property 'NUM' not allowed on 'i'
  self._init_synopsis(pred, synopsis_data, propcache))
/Users/lizcconrad/.venvs/pogg_tests/lib/python3.13/site-packages/delphin/semi.py:471: SemIWarning: one+less_a: property 'NUM' not allowed on 'i'
  self._init_synopsis(pred, synopsis_data, propcache))
/Users/lizcconrad/.venvs/pogg_tests/lib/python3.13/site-packages/delphin/semi.py:471: SemIWarning: one+more_a: property 'NUM' not allowed on 'i'
  self._init_synopsis(pred, synopsis_data, propcache))
/Users/lizcconrad/.venvs/pogg_tests/lib/python3.13/site-packages/delphin/semi.py:471: SemIWarning: poss: property 'NUM' not allowed on 'i'
  self._init_synopsis(pred, synopsis_data, propcache))


In [96]:


# make SEMENTs for "Liz," "eat," "a," "tasty," and "cookie"
liz = sem_algebra.create_CARG_SEMENT(pogg_config, "named", "Liz")
eat = sem_algebra.create_base_SEMENT(pogg_config, "_eat_v_1")
a = sem_algebra.create_base_SEMENT(pogg_config, "_a_q")
tasty = sem_algebra.create_base_SEMENT(pogg_config, "_tasty_a_1")
cookie = sem_algebra.create_base_SEMENT(pogg_config, "_cookie_n_1")


In [97]:

sements = [liz, eat, a, tasty, cookie]
for s in sements:
    print("{}\n".format(sementcodecs.encode(s, indent=True)))

[ TOP: h2
  INDEX: x1
  RELS: < [ named LBL: h2 ARG0: x1 CARG: "Liz" ] > ]

[ TOP: h6
  INDEX: e3
  RELS: < [ _eat_v_1 LBL: h6 ARG0: e3 ARG1: i4 ARG2: i5 ] >
  SLOTS: < ARG1: i4 ARG2: i5 > ]

[ TOP: h11
  INDEX: x7
  RELS: < [ _a_q LBL: h10 ARG0: x7 RSTR: h8 BODY: h9 ] >
  SLOTS: < ARG0: x7 RSTR: h8 BODY: h9 > ]

[ TOP: h14
  INDEX: e12
  RELS: < [ _tasty_a_1 LBL: h14 ARG0: e12 ARG1: u13 ] >
  SLOTS: < ARG1: u13 > ]

[ TOP: h16
  INDEX: x15
  RELS: < [ _cookie_n_1 LBL: h16 ARG0: x15 ] > ]



In [98]:
tasty_cookie = sem_algebra.op_non_scopal_argument_hook(tasty, cookie, "ARG1")
print(sementcodecs.encode(tasty_cookie, indent=True))

[ TOP: h16
  INDEX: x15
  RELS: < [ _tasty_a_1 LBL: h14 ARG0: e12 ARG1: u13 ]
          [ _cookie_n_1 LBL: h16 ARG0: x15 ] >
  EQS: < h14 eq h16 u13 eq x15 > ]
